## 从CSV加载图形

In [1]:
from torch_geometric.data import download_url, extract_zip

url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
extract_zip(download_url(url, '.'), '.')

movie_path = './ml-latest-small/movies.csv'
rating_path = './ml-latest-small/ratings.csv'

Using exist file ml-latest-small.zip
Extracting ./ml-latest-small.zip


In [2]:
import pandas as pd

print(pd.read_csv(movie_path).head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


&emsp;&emsp;我们看到该`movies.csv`文件提供了三列：`movieId`为每部电影分配一个唯一标识符，而`title`和`genres`列代表给定电影的标题和流派。

&emsp;&emsp;我们可以利用`title`和`genres`这两列来定义一个可以被机器学习模型轻松解释的特征表示。

In [3]:
print(pd.read_csv(rating_path).head())

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


&emsp;&emsp;`ratings.csv`数据连接用户（由`userId`给出）和电影（由`movieId`给出），并定义给定用户如何对特定电影进行评分（`rating`）。 为简单起见，我们不使用额外的时间戳信息（`timestamp`）。

&emsp;&emsp;为了以`PyG`数据格式表示此数据，我们首先定义了一个方法`load_node_csv()`, 该方法读取`*.csv`文件并返回形状为`[num_nodes, num_features]`的节点级特征表示`x`。

In [4]:
import torch

def load_node_csv(path, index_col, encoders=None, **kwargs):
    df = pd.read_csv(path, index_col=index_col, **kwargs)
    """
    df=pd.read_csv(rating_path).head(5)
    df.index # RangeIndex(start=0, stop=5, step=1)
    df.index.unique() # Int64Index([0, 1, 2, 3, 4], dtype='int64')
    """
    mapping = {index: i for i, index in enumerate(df.index.unique())}

    x = None
    if encoders is not None:
        # encoders.itmes() 每次迭代返回元组对象(key,element)
        xs = [encoder(df[col]) for col, encoder in encoders.items()]
        x = torch.cat(xs, dim=-1)

    return x, mapping

&emsp;&emsp;此处，`load_node_csv()`从路径读取`*.csv`文件，并创建一个字典映射，将其索引列映射到`{ 0, ..., num_rows - 1 }`范围内的连续值。这是必需的，因为我们希望我们的最终数据表示尽可能紧凑，例如，第一行中电影的表示应该可以通过`x[0]`访问。

&emsp;&emsp;我们进一步利用了编码器的概念，它定义了如何将特定列的值编码为数字特征表示。例如，我们可以定义一个句子编码器，将原始列字符串编码为低维嵌入。为此，我们利用了优秀的句子转换器库，该库提供了大量最先进的预训练`NLP`嵌入模型：

```bash
pip install sentence-transformers

```

In [6]:
from sentence_transformers import SentenceTransformer

class SequenceEncoder(object):
    def __init__(self, model_name='all-MiniLM-L6-v2', device=None):
        self.device = device
        self.model = SentenceTransformer(model_name, device=device)

    @torch.no_grad() # 禁用梯度, 前向计算节省显存
    def __call__(self, df):
        x = self.model.encode(df.values, show_progress_bar=True,
                              convert_to_tensor=True, device=self.device)
        return x.cpu()


ModuleNotFoundError: No module named 'sentence_transformers'

&emsp;&emsp;`SequenceEncoder`类加载由`model_name`给定的预训练`NLP`模型，并使用它将字符串列表编码为形状为`[num_strings, embedding_dim]`的 `PyTorch`张量。 我们可以使用这个`SequenceEncoder`来编码`movies.csv`文件的标题；

&emsp;&emsp;以类似的方式，我们可以创建另一个编码器，将电影类型（例如 Adventure|Children|Fantasy）转换为分类标签。 为此，我们首先需要找到数据中存在的所有现有流派，创建形状为`[num_movies, num_genres]`的特征表示`x`，并为`x[i, j]`分配`1`，以表示电影`i`中存在流派`j` ：

In [7]:
class GenresEncoder(object):
    def __init__(self, sep='|'):
        self.sep = sep

    def __call__(self, df):
        genres = set(g for col in df.values for g in col.split(self.sep))
        mapping = {genre: i for i, genre in enumerate(genres)}

        x = torch.zeros(len(df), len(mapping)) # [num_movies, num_genres]
        for i, col in enumerate(df.values):
            for genre in col.split(self.sep):
                x[i, mapping[genre]] = 1
        return x

&emsp;&emsp;有了这个，我们可以通过以下方式获得电影的最终表示：

In [8]:
movie_x, movie_mapping = load_node_csv(
    movie_path, index_col='movieId', encoders={
        'title': SequenceEncoder(),
        'genres': GenresEncoder()
    })

NameError: name 'SequenceEncoder' is not defined

&emsp;&emsp;同样，我们也可以利用 load_node_csv() 来获取从 userId 到连续值的用户映射。 但是，此数据集中的用户没有其他特征信息。 因此，我们没有定义任何编码器：

In [9]:
_, user_mapping = load_node_csv(rating_path, index_col='userId')

&emsp;&emsp;有了这个，我们准备初始化我们的 HeteroData 对象并将两种节点类型传递给它：

In [10]:
from torch_geometric.data import HeteroData

data = HeteroData()

data['user'].num_nodes = len(user_mapping)  # Users do not have any features.
data['movie'].x = movie_x

print(data)
"""
HeteroData(
  user={ num_nodes=610 },
  movie={ x[9742, 404] }
)
"""


ImportError: cannot import name 'HeteroData' from 'torch_geometric.data' (/Users/hezhiqiang01/Desktop/anaconda/anaconda3/envs/ecole/lib/python3.9/site-packages/torch_geometric/data/__init__.py)

&emsp;&emsp;由于用户没有任何节点级别的信息，我们仅定义其节点数。 因此，在训练异构图模型期间，我们可能需要通过`torch.nn.Embedding`以端到端的方式学习不同的用户嵌入。

&emsp;&emsp;接下来，我们看看将用户与他们的评级定义的电影联系起来。 为此，我们定义了一个方法`load_edge_csv()`，该方法从`ratings.csv`返回形状`[2, num_ratings]`的最终`edge_index`表示，以及原始`*.csv`文件中存在的任何其他特征：

In [11]:
def load_edge_csv(path, src_index_col, src_mapping, dst_index_col, dst_mapping,
                  encoders=None, **kwargs):
    df = pd.read_csv(path, **kwargs)

    src = [src_mapping[index] for index in df[src_index_col]]
    dst = [dst_mapping[index] for index in df[dst_index_col]]
    edge_index = torch.tensor([src, dst])

    edge_attr = None
    if encoders is not None:
        edge_attrs = [encoder(df[col]) for col, encoder in encoders.items()]
        edge_attr = torch.cat(edge_attrs, dim=-1)

    return edge_index, edge_attr